# Discovering Lyon (France)

## 1. Introduction

### 1.1 Background

Lyon is one of the main French cities. It is particularly famous for its cuisine and gastronomy as well as for its historical and architectural landmarks.
Half a million people live there and every year thousands of new citizens from all over the world move in the city.

### 1.2 Problem

Have you ever moved to a city you did not know ? It can be a real headache to find out which neighborhood would be best for you depending on your hobbies.
This project will describe Lyon's boroughs in order to help new citizens to find out which one would be best for them.

## 2. Data

For this study, we will use the following data from Wikipédia that represents the different boroughs of Lyon. We will need to put the data into a dataframe and to find out the location of all boroughs using Google Maps. 
The following list has been written down into a text file called boroughslyon.txt where the first line is the borough followed by all neiborhoods in the borough line by line. Boroughs are seperated by line breaks.

    1st arrondissement: Slopes of La Croix-Rousse, Terreaux, Martinière/St-Vincent
    2nd arrondissement: Cordeliers, Bellecour, Ainay, Perrache, Confluence, Sainte-Blandine
    3rd arrondissement: Guillotière (north), Préfecture, Part-Dieu, Villette, Dauphiné/Sans Souci, Montchat, Grange Blanche (north), Monplaisir (north)
    4th arrondissement: Plateau de la Croix-Rousse, Serin
    5th arrondissement: Vieux Lyon (Saint-Paul, Saint-Jean, Saint-Georges), Saint-Just, Saint-Irénée, Fourvière, Point du Jour, Ménival, Battières, Champvert (south)
    6th arrondissement: Brotteaux, Bellecombe, Parc de la Tête d'or, Cité Internationale
    7th arrondissement: Guillotière (south), Jean Macé, Gerland
    8th arrondissement: Monplaisir (south), Bachut, États-Unis, Grand Trou/Moulin à Vent, Grange Blanche (south), Laënnec, Mermoz, Monplaisir-la-Plaine
    9th arrondissement: Vaise, Duchère, Rochecardon, St-Rambert-l'Île-Barbe, Gorge de Loup, Observance, Champvert (north)

We will then use the Foursquare's API to find out about the main venues of these boroughs and we will try to determine the main kinds of activities that can be done in each boroughs.
Concerning the Foursquare data, we will focus on the best venues for each neiborhood.

In [1]:
# import statements
import pandas as pd

# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium 

import requests
from pandas.io.json import json_normalize 

Creating the dataframe from the boroughslyon.txt text file.

In [2]:
# read boroughslyon.txt
file = open('boroughslyon.txt')
read = file.readlines()

# create the list from the text file
data = []
borough=''
neighborhood=''
for i, line in enumerate(read):
    if (i==0):
        borough=line
    elif (read[i - 1]=='\n'):
        borough=line
    elif (line!='\n'):
        neighborhood=line
        if borough.endswith('\n'):
            borough = borough[:-1]
        if neighborhood.endswith('\n'):
            neighborhood = neighborhood[:-1]
        data.append([neighborhood,borough])
data

[['Pentes de la Croix-Rousse', '1st arrondissement'],
 ['Terreaux', '1st arrondissement'],
 ['Martinière/Saint-Vincent', '1st arrondissement'],
 ['Cordeliers', '2nd arrondissement'],
 ['Bellecour', '2nd arrondissement'],
 ['Ainay', '2nd arrondissement'],
 ['Perrache', '2nd arrondissement'],
 ['Confluence', '2nd arrondissement'],
 ['Sainte-Blandine', '2nd arrondissement'],
 ['La Guillotière', '3rd arrondissement'],
 ['Préfecture', '3rd arrondissement'],
 ['Part-Dieu', '3rd arrondissement'],
 ['La Villette', '3rd arrondissement'],
 ['Dauphiné/Sans Souci', '3rd arrondissement'],
 ['Montchat', '3rd arrondissement'],
 ['Grange Blanche', '3rd arrondissement'],
 ['Monplaisir', '3rd arrondissement'],
 ['Croix-Rousse', '4th arrondissement'],
 ['Serin', '4th arrondissement'],
 ['Vieux Lyon', '5th arrondissement'],
 ['Saint-Just', '5th arrondissement'],
 ['Saint-Irénée', '5th arrondissement'],
 ['Fourvière', '5th arrondissement'],
 ['Point du Jour', '5th arrondissement'],
 ['Ménival', '5th arrond

Let's find latitude and longitude for each neighborhood

In [5]:
geolocator = Nominatim(user_agent="lyon_agent")
ll=[]
for i, neighborhood in enumerate(data):
    address = data[i][0] + ', Lyon, France'
    try:
        location = geolocator.geocode(address)
        ll.append([data[i][0], data[i][1], location.latitude, location.longitude])
    except:
        print('Error:', neighborhood)
df = pd.DataFrame(ll, columns = ['Neighborhood', 'Borough', 'Latitude', 'Longitude'])

df.head()

,Neighborhood,Borough,Latitude,Longitude
0,Pentes de la Croix-Rousse,1st arrondissement,45.771915,4.830748
1,Terreaux,1st arrondissement,45.768442,4.832414
2,Martinière/Saint-Vincent,1st arrondissement,45.768221,4.827626
3,Cordeliers,2nd arrondissement,45.763129,4.835797
4,Bellecour,2nd arrondissement,45.757362,4.833009


Let's group neighborhoods by boroughs


In [260]:
df = df.groupby(['Neighborhood','Latitude','Longitude'],sort=False)['Borough'].apply(', '.join).reset_index()
df

,Neighborhood,Latitude,Longitude,Borough
0,Pentes de la Croix-Rousse,45.771915,4.830748,1st arrondissement
1,Terreaux,45.768442,4.832414,1st arrondissement
2,Martinière/Saint-Vincent,45.768221,4.827626,1st arrondissement
3,Cordeliers,45.763129,4.835797,2nd arrondissement
4,Bellecour,45.757362,4.833009,2nd arrondissement
5,Ainay,45.753903,4.828743,2nd arrondissement
6,Perrache,45.749542,4.827075,2nd arrondissement
7,Confluence,45.740666,4.818047,2nd arrondissement
8,Sainte-Blandine,45.744204,4.821978,2nd arrondissement
9,La Guillotière,45.752698,4.846050,"3rd arrondissement, 7th arrondissement"


Let's create a map to visualise all neighborhoods

In [7]:
# create map of Lyon using latitude and longitude values
latitude = 45.76
longitude = 4.84
map_lyon = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=500,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lyon)  
    
map_lyon

Getting JSON from Foursquare

In [55]:
CLIENT_ID = 'P552ZMYN1TOL2XXNNWJ2HL3CG0PPDORKK3H2SVCJCTS0GACT' # your Foursquare ID
CLIENT_SECRET = '0BI1A1HGVDEDREA2Y0BDWDX1QNY5DSIKPBHOOWUWYWERLSUC' # your Foursquare Secret
VERSION = '20190930'

In [206]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=10):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        category1 = '4d4b7104d754a06370d81259' # Culture and leisure
        category2 = '4d4b7105d754a06372d81259' # University facilities
        category3 = '4d4b7105d754a06373d81259' # Events
        category4 = '4d4b7105d754a06376d81259' # Partying
        category5 = '4d4b7105d754a06377d81259' # Outdoor and leisure
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [207]:
lyon_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Pentes de la Croix-Rousse
Terreaux
Martinière/Saint-Vincent
Cordeliers
Bellecour
Ainay
Perrache
Confluence
Sainte-Blandine
La Guillotière
Préfecture
Part-Dieu
La Villette
Dauphiné/Sans Souci
Montchat
Grange Blanche
Monplaisir
Croix-Rousse
Serin
Vieux Lyon
Saint-Just
Saint-Irénée
Fourvière
Point du Jour
Ménival
Battières
Champvert
Brotteaux
Bellecombe
Parc de la Tête d'or
Cité Internationale
Jean Macé
Gerland
Bachut
États-Unis
Grand Trou
Moulin à Vent
Laënnec
Mermoz
Monplaisir-la-Plaine
Vaise
Duchère
Rochecardon
Saint-Rambert
Île-Barbe
Gorge de Loup
Observance


Modify some Categories names in order to have better clusters

In [258]:
lyon_venues = lyon_venues.replace('French Restaurant', 'Restaurant')
lyon_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Pentes de la Croix-Rousse,45.771915,4.830748,Balthaz'art,45.773297,4.832122,Restaurant
1,Pentes de la Croix-Rousse,45.771915,4.830748,Place de l'Esplanade,45.772420,4.831818,Plaza
2,Pentes de la Croix-Rousse,45.771915,4.830748,Le Desjeuner,45.773597,4.832150,Restaurant
3,Pentes de la Croix-Rousse,45.771915,4.830748,Café J'Adore,45.771290,4.831567,Café
4,Pentes de la Croix-Rousse,45.771915,4.830748,Pentes de la Croix Rousse,45.771141,4.828272,Neighborhood


In [212]:
# one hot encoding
lyon_onehot = pd.get_dummies(lyon_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lyon_onehot['Neighborhoods'] = lyon_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lyon_onehot.columns[-1]] + list(lyon_onehot.columns[:-1])
lyon_onehot = lyon_onehot[fixed_columns]

lyon_onehot.head()

,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,...,Trail,Train Station,Tram Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Zoo
0,Pentes de la Croix-Rousse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Pentes de la Croix-Rousse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Pentes de la Croix-Rousse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Pentes de la Croix-Rousse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Pentes de la Croix-Rousse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [257]:
lyon_grouped = lyon_onehot.groupby('Neighborhoods').mean().reset_index()
lyon_grouped.head()

,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,...,Trail,Train Station,Tram Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Zoo
0,Ainay,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
1,Bachut,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Battières,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bellecombe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bellecour,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [261]:
import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = lyon_grouped['Neighborhoods']

for ind in np.arange(lyon_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lyon_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Ainay,New American Restaurant,Restaurant,Wine Bar,Art Museum,Cupcake Shop
1,Bachut,Food & Drink Shop,Bus Stop,Supermarket,Grocery Store,Comedy Club
2,Battières,Pizza Place,Supermarket,Recording Studio,Basketball Court,Bus Stop
3,Bellecombe,Restaurant,Nightclub,Japanese Restaurant,Diner,Bar
4,Bellecour,Plaza,Dessert Shop,Hotel,Café,Bookstore


In [249]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

lyon_grouped_clustering = lyon_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lyon_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 1, 1, 0, 4, 0, 1, 2, 2, 4, 0, 1, 4, 4, 0, 2, 4, 0, 0, 0, 2, 2,
       4, 1, 0, 0, 4, 1, 1, 2, 0, 0, 0, 0, 0, 2, 0, 1, 1, 3, 2, 0, 4, 4,
       4, 0, 3], dtype=int32)

In [262]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
final_df = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
final_df = final_df.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhood')

final_df.head()

,Neighborhood,Latitude,Longitude,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Pentes de la Croix-Rousse,45.771915,4.830748,1st arrondissement,0,Restaurant,Plaza,Neighborhood,Bakery,Café
1,Terreaux,45.768442,4.832414,1st arrondissement,4,Bar,Japanese Restaurant,Cocktail Bar,Plaza,Café
2,Martinière/Saint-Vincent,45.768221,4.827626,1st arrondissement,4,Bar,Cocktail Bar,Restaurant,Wine Bar,Public Art
3,Cordeliers,45.763129,4.835797,2nd arrondissement,4,Plaza,Lyonese Bouchon,Bar,Burger Joint,Café
4,Bellecour,45.757362,4.833009,2nd arrondissement,4,Plaza,Dessert Shop,Hotel,Café,Bookstore


In [251]:
import folium
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

latitude = 45.76
longitude = 4.84

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_df['Latitude'], final_df['Longitude'], final_df['Neighborhood'], final_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [252]:
final_df.loc[final_df['Cluster Labels'] == 0, final_df.columns[[0] + list(range(5, final_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Pentes de la Croix-Rousse,Restaurant,Plaza,Neighborhood,Bakery,Café
6,Perrache,Restaurant,Plaza,Music Venue,Indian Restaurant,Coffee Shop
9,La Guillotière,Bistro,Hostel,Restaurant,Bar,Hobby Shop
11,Part-Dieu,Restaurant,Wine Shop,Indian Restaurant,Concert Hall,Hotel
15,Grange Blanche,Restaurant,Museum,Metro Station,Garden,Bakery
16,Monplaisir,Museum,Restaurant,Food & Drink Shop,Plaza,Brasserie
17,Croix-Rousse,Restaurant,Pub,Plaza,Farmers Market,Street Art
18,Serin,River,Home Service,Park,Outdoors & Recreation,Restaurant
23,Point du Jour,Restaurant,Optical Shop,Wings Joint,Paper / Office Supplies Store,Farmers Market
27,Brotteaux,Restaurant,Italian Restaurant,Train Station,Japanese Restaurant,Speakeasy


Cluster 0: Lively neighborhoods in different places in Lyon. There are restaurants, plazas, bars and other places to go out and do activities with friends. It is perfect for people wanting to stay in a very dynamic neiborhood and liking to go out with friends.

In [253]:
final_df.loc[final_df['Cluster Labels'] == 1, final_df.columns[[0] + list(range(5, final_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,Dauphiné/Sans Souci,Food & Drink Shop,Bus Stop,Bar,Bakery,Plaza
20,Saint-Just,Bus Stop,Lawyer,Hotel,Plaza,Cable Car
21,Saint-Irénée,Bus Stop,Arts & Entertainment,Fast Food Restaurant,Zoo,Garden
24,Ménival,Business Service,Bus Stop,Pizza Place,Supermarket,Zoo
25,Battières,Pizza Place,Supermarket,Recording Studio,Basketball Court,Bus Stop
26,Champvert,Playground,Pizza Place,Food & Drink Shop,Tennis Court,Grocery Store
33,Bachut,Food & Drink Shop,Bus Stop,Supermarket,Grocery Store,Comedy Club
36,Moulin à Vent,Bus Stop,Turkish Restaurant,Pizza Place,Soccer Stadium,Zoo
38,Mermoz,Bus Stop,Kebab Restaurant,Pizza Place,Gas Station,Metro Station


Cluster 1: Bus Stops are the most important venues of these neiborhoods. We notice as well that they are not in the center of Lyon. Main venues are supermarkets, a few restaurants and sport facilities. This places seem to be very residential and perfect for families or people looking for some quiet.

In [254]:
final_df.loc[final_df['Cluster Labels'] == 2, final_df.columns[[0] + list(range(5, final_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,Confluence,Fast Food Restaurant,Electronics Store,Art Gallery,Deli / Bodega,Concert Hall
8,Sainte-Blandine,Hotel,Multiplex,Playground,Concert Hall,Deli / Bodega
10,Préfecture,Hotel,Lyonese Bouchon,Bar,Chinese Restaurant,Plaza
12,La Villette,Hotel,Italian Restaurant,Sandwich Place,Supermarket,Diner
30,Cité Internationale,Restaurant,Hotel,Concert Hall,Coffee Shop,Café
37,Laënnec,Hotel,Auto Dealership,Zoo,Food & Drink Shop,Convenience Store
45,Gorge de Loup,Gas Station,Hotel,Flower Shop,Train Station,Motel
46,Observance,Train Station,Hotel,Motel,Hostel,Dessert Shop


Cluster 2: Many hotels and train stations in these neiborhoods as well as food places. They are perfect neiborhoods for tourist staying a few days or business travelers.

In [255]:
final_df.loc[final_df['Cluster Labels'] == 3, final_df.columns[[0] + list(range(5, final_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
43,Saint-Rambert,Restaurant,Bakery,Supermarket,Flower Shop,Concert Hall
44,Île-Barbe,Restaurant,Bakery,Garden,Convenience Store,Cupcake Shop


These 2 neighborhoods are very close and some venues are probably the same for both. It seems to be a perfect place for people liking nature and food but it is hard to tell more about it as there are only 2 neighborhoods...

In [256]:
final_df.loc[final_df['Cluster Labels'] == 4, final_df.columns[[0] + list(range(5, final_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Terreaux,Bar,Japanese Restaurant,Cocktail Bar,Plaza,Café
2,Martinière/Saint-Vincent,Bar,Cocktail Bar,Restaurant,Wine Bar,Public Art
3,Cordeliers,Plaza,Lyonese Bouchon,Bar,Burger Joint,Café
4,Bellecour,Plaza,Dessert Shop,Hotel,Café,Bookstore
5,Ainay,New American Restaurant,Restaurant,Wine Bar,Art Museum,Cupcake Shop
14,Montchat,Plaza,Pool,Supermarket,Park,Athletics & Sports
19,Vieux Lyon,Lyonese Bouchon,Ice Cream Shop,Church,Beer Bar,Café
22,Fourvière,Historic Site,Lyonese Bouchon,Church,Neighborhood,Park
35,Grand Trou,Plaza,Rental Car Location,Auto Workshop,Zoo,Food & Drink Shop
40,Vaise,Train Station,Auto Workshop,Department Store,Electronics Store,Food & Drink Shop


Cluster 4: A lot of plazas as well as bars and restaurants. While cluster 0 was more about going out, this cluster seems to be more about sitting down for a coffee or a brunch in a nice place as French people love to do.